This note book is designed to generate a simplified framework for the calculation of the GMDB and GMIB.

 SETTING-UP the Framework:

1): Code to read N asset scenario files (CSV) from the disk.
The code  starts by defining a variable that represents the asset classes that we want to load from the disk:

 2): Add to the code the definition of policy information: Issue age, Gender


 3): The definition of the GMDB : Rollup, reset, fees

 4): Same for GMIB

5): Calculate the duration of the accumulation phase, 
    #this will determine what portion of the scenario files must be extracted.

 NB: the original data is: 10,000 x 360.

But the project runs only 1,000 scenarios for 240 months,then the code should only extract 1,000 x 241 data points from each asset scenario file.

In [64]:
import pandas as pd
import numpy as np
import os.path

###....................................................................

class Generic_Parameters:
    pass

In [65]:
## 1.Initializing the GMDB and GMIB features of the sample variable annuity contract##

GMDB_RIDER_FEE         = 120     ## bps
GMDB_ROLLUP_RATE       = 3.5/100  ## 3.5%
GMDB_RESET_FREQUENCY   = 18       ## months

###.......................................................................
GMIB_RIDER_FEE         = 150     ## bps
GMIB_ROLLUP_RATE       = 4.0/100  ## 4%
GMIB_RESET_FREQUENCY   = 12       ## months

GMIB_WAIT_PERIOD       = 10       ##years
GMIB_MIN_AGE           = 60       ##years
GMIB_MAX_AGE           = 70       ##years
###........................................................................

In [66]:
## 2. Sample Policy information

ISSUE_AGE              = 50
INITIAL_PREMIUM        = 1000
###.........................................................................

ASSET_CLASS            = ["FIXED", ## Diversified Fixed Income
                         "BALANCED", ## Diversified Balanced Allocation
                         "SMALL"]    ## Intermediate Risk Equity

NUM_FUNDS              = len(ASSET_CLASS)

## 20% in FUND-1,30% in FUND-2, 50% in FUND-3

FUND_WEIGHT            = [0.20,0.30,0.50]  


In [67]:
## 3. LOCATIONS OF THE SCENARIO FILES IN THE DISK
# One can customize the location of the scenarios depending on where they saved them

SCENARIOS_FILES_LOCATION = "/Users/promise/Documents/Scenarios_File/"

## Number of  scenarios projected for each asset class

NUM_SCENARIOS                = 1000

###............................................................................


In [68]:
## 4. Global parameters to define an object called gmib_rider, that will be used in the simulation

gmib_rider                 = Generic_Parameters()
####
gmib_rider.fee             = GMIB_RIDER_FEE
gmib_rider.rollup_rate     = GMIB_ROLLUP_RATE

gmib_rider.reset_frequency = GMIB_RESET_FREQUENCY
gmib_rider.wait_period     = GMIB_WAIT_PERIOD
gmib_rider.min_age         = GMIB_MIN_AGE
gmib_rider.max_age         = GMIB_MAX_AGE


In [69]:
## 5. Printing the GMIB rider

line_sep = str.join('', ['='*40])
print(line_sep)
print("GMIB benefit rider for the VA contract")
print(line_sep)
print("Rider fee: %5d" %(gmib_rider.fee), 'bps')
print("Rollup rate : %5.3f" % (gmib_rider.rollup_rate), "(%.1f" %(gmib_rider.rollup_rate*100) + "%) APR")
print("Reset frequency: %5d" %(gmib_rider.reset_frequency), 'month(s)') 
print(line_sep)
print("Wait period before annuitization:",gmib_rider.wait_period)
print("Minimum age for annuitization :",gmib_rider.min_age)
print("Maximum age for annuitization :",gmib_rider.max_age)

GMIB benefit rider for the VA contract
Rider fee:   150 bps
Rollup rate : 0.040 (4.0%) APR
Reset frequency:    12 month(s)
Wait period before annuitization: 10
Minimum age for annuitization : 60
Maximum age for annuitization : 70


In [70]:
##6. Global parameters to define an object called gmdb_rider, that will be used in the simulations

gmdb_rider                = Generic_Parameters()
gmdb_rider.fee            = GMDB_RIDER_FEE
gmdb_rider.rollup_rate    = GMDB_ROLLUP_RATE  


gmdb_rider.reset_frequency = GMDB_RESET_FREQUENCY
##..................................................
line_sep = str.join('', ['='*40])
print(line_sep)
print("GMDB benefit rider for the VA contract")
print(line_sep)
print("Rider fee : %5d" %(gmdb_rider.fee), 'bps')
print("Rollup rate : %5.3f" %(gmdb_rider.rollup_rate), 
      "(%.1f" %(gmdb_rider.rollup_rate*100) + "%) APR")
print("Reset frequency: %5d" %(gmdb_rider.reset_frequency), 'month(s)')

GMDB benefit rider for the VA contract
Rider fee :   120 bps
Rollup rate : 0.035 (3.5%) APR
Reset frequency:    18 month(s)


In [71]:
## 7. Defining sample policy

sample_policy                 = Generic_Parameters()
sample_policy.issue_age       = ISSUE_AGE
sample_policy.initial_premium = INITIAL_PREMIUM
sample_policy.num_funds       = NUM_FUNDS
sample_policy.asset_class     = ASSET_CLASS
sample_policy.fund_weight     = FUND_WEIGHT

## minimum age at which investor may annuitize the contract 
sample_policy.min_gmib_age    =  max(sample_policy.issue_age + gmib_rider.wait_period, gmib_rider.min_age)


In [72]:
#8 Printing the sample policy
print(line_sep)
print("Sample VA contract with GMDB and GMIB")
print(line_sep)
print("Issue age :",sample_policy.issue_age)
print("Minimum annuitization age:",sample_policy.min_gmib_age)
print("Maximum annuitization age:",gmib_rider.max_age)
print(line_sep)
print("Asset (investment fund) allocatio")
for k in range(sample_policy.num_funds):
    print("%8s: %d"%(sample_policy.asset_class[k],sample_policy.fund_weight[k]*1000)+'%')

Sample VA contract with GMDB and GMIB
Issue age : 50
Minimum annuitization age: 60
Maximum annuitization age: 70
Asset (investment fund) allocatio
   FIXED: 200%
BALANCED: 300%
   SMALL: 500%


In [73]:
## 9. parameters of the accumulation phase

In [74]:
accum_phase_params               =Generic_Parameters()
accum_phase_params.num_years     = gmib_rider.max_age - sample_policy.issue_age 
accum_phase_params.num_months    = accum_phase_params.num_years * 12
accum_phase_params.num_scenarios = NUM_SCENARIOS

print(line_sep)
print("Parameters of the accumulation phase")
print (line_sep)
print("Number of projection years : %4d" % (accum_phase_params.num_years))
print("Number of projection months : %4d" % (accum_phase_params.num_months)) 
print("Number of scenarios projected: %4d" % (accum_phase_params.num_scenarios))

Parameters of the accumulation phase
Number of projection years :   20
Number of projection months :  240
Number of scenarios projected: 1000


In [75]:
## 10. Global Simulation Parameters

simul_params                       = Generic_Parameters()
simul_params.num_funds             = NUM_FUNDS

simul_params.asset_class           = ASSET_CLASS

simul_params.num_scenarios         = NUM_SCENARIOS

simul_params.accum_phase           = accum_phase_params
simul_params.data_size             = Generic_Parameters
simul_params.data_size.num_rows    = simul_params.num_scenarios
simul_params.data_size.num_columns = accum_phase_params.num_months + 1
simul_params.data_size.matrix_dim  = \
(simul_params.data_size.num_rows, simul_params.data_size.num_columns)
simul_params.timestep              =  Generic_Parameters()
simul_params.timestep.dt           = 1.0/12



In [76]:
## 11. Printing Global simulation parameters

print(line_sep +'\n' + "simulation parameters")
print(line_sep)
print("Number of investment funds:",simul_params.num_funds)
print("Asset classes too project :",str.join(',',simul_params.asset_class))
print("Number of scenarios to project :",simul_params.num_scenarios)
print(line_sep)
print("Number of rows in matrix variables :%5d" %(simul_params.data_size.num_rows))
print("Number of columns in matrix variables :%5d" %(simul_params.data_size.num_columns))
print("Simulation time-step(is monthly) :%.3f" %(simul_params.timestep.dt))

simulation parameters
Number of investment funds: 3
Asset classes too project : FIXED,BALANCED,SMALL
Number of scenarios to project : 1000
Number of rows in matrix variables : 1000
Number of columns in matrix variables :  241
Simulation time-step(is monthly) :0.083


In [77]:
## 12. Defining additional parameters

num_columns = simul_params.data_size.num_columns 
for gftype in ["gmdb", "gmib"]:
    gf_rider = gmdb_rider if (gftype == "gmdb") else gmib_rider 

    gf_params = Generic_Parameters()

    rider_fee = gf_rider.fee * 1.0/10000 ## bips to (%)
    gf_params.rider_fee_factor = 1 - np.exp(-rider_fee * simul_params.timestep.dt)
                                           
    rup_rate = gf_rider.rollup_rate


    gf_params.rollup_factor = np.exp(rup_rate * simul_params.timestep.dt)
    gf_params.is_reset_date = np.array([False] *num_columns)
    gf_params.reset_dates = None
    reset_freq  = gf_rider.reset_frequency
    if (reset_freq > 0):
        gf_params.reset_dates = [k for k in range(0,num_columns, reset_freq)]
        for ti in gf_params.reset_dates:
             gf_params.is_reset_date [ti] = True
    if (gftype == "gmdb"):
        simul_params.gmdb = gf_params
    else:
        simul_params.gmib = gf_params


In [78]:
## 13. Printing the value of:

#GMDB rollup factor
#GMDB rider factor
#Reset months

# NOTE: The reset frequency for GMDB is 18 months


In [79]:
print(line_sep + '\n' + "Simulation parameters - GMDB benefit rider")
print(line_sep)
print("Rider fee : %9d"  %(gmdb_rider.fee), 'bps')
print("Rider fee factor: %.7f" %(simul_params.gmdb.rider_fee_factor), 'monthly')
print("Rollup rate : %6.5f" %(gmdb_rider.rollup_rate), "(%.1F" %(gmdb_rider.rollup_rate*100) + '%) APR')
print("Rollup factor : %6.5f" %(simul_params.gmdb.rollup_factor), 'monthly')
print("Reset frequency : %9d"  %(gmdb_rider.reset_frequency), 'month(s)')
print(line_sep + '\n' + "Reset dates (months)")
print(str.join(', ',[str(x) for x in simul_params.gmdb.reset_dates]))

Simulation parameters - GMDB benefit rider
Rider fee :       120 bps
Rider fee factor: 0.0009995 monthly
Rollup rate : 0.03500 (3.5%) APR
Rollup factor : 1.00292 monthly
Reset frequency :        18 month(s)
Reset dates (months)
0, 18, 36, 54, 72, 90, 108, 126, 144, 162, 180, 198, 216, 234


In [80]:
#14. ## 13. Printing the value of:

#GMIB rollup factor
#GMIB rider factor
#Reset months

# NOTE: The reset frequency for GMDB is 12 months


In [81]:
print(line_sep + '\n' + "Simulation parameters - GMIB benefit rider")
print(line_sep)

print("Rider fee : %9d"  %(gmib_rider.fee), 'bps')
print("Rider fee factor : %.7f" %(simul_params.gmib.rider_fee_factor), "monthly")
print("Rollup rate : %6.5f " %(gmib_rider.rollup_rate), "(%.1F" %(gmib_rider.rollup_rate*100) + '%) APR')

print("Rollup factor : %6.5f" %(simul_params.gmib.rollup_factor), "monthly")
print("Reset frequency : %od"  %(gmib_rider.reset_frequency), "month(s)")
print(line_sep + '\n' + "Reset dates (months)")

print(str.join(",",[str(x) for x in simul_params.gmib.reset_dates]))

Simulation parameters - GMIB benefit rider
Rider fee :       150 bps
Rider fee factor : 0.0012492 monthly
Rollup rate : 0.04000  (4.0%) APR
Rollup factor : 1.00334 monthly
Reset frequency : 14d month(s)
Reset dates (months)
0,12,24,36,48,60,72,84,96,108,120,132,144,156,168,180,192,204,216,228,240


In [82]:
## 15. Function1 : Load asset scenario files and extraxt only the data points needed to run the simulations

def load_asset_scenarios (location, asset_class,n_scenarios,n_months):
    ## Name of the file to load 
    filename =  location  + asset_class+ ".csv"

    print("Scenarios file := ", filename)
## Check if the scenario file really exists on the disk fexists = os.path.isfile(filename)
    fexists=os.path.isfile(filename)
    if not (fexists):
        print(">>** ERROR **: file [DOES NOT EXIST]") 
        return None ## No scenarios to return to the user
    print("Loading asset scenarios file from disk ...") 
    csv_outputs = pd.read_csv(filename, header= None) 
    all_scenarios = csv_outputs.values
    outputs = all_scenarios[:n_scenarios, : n_months+1]
  
#################################################
    print("DONE: loading asset scenarios") 
    return outputs


In [83]:
## 16. Load the asset scenarios for the three(3) selected

line_sep = str.join('', ['='*40])
## ........................................................
print("Asset classes: ", str.join(",",simul_params.asset_class)) 
print(line_sep)
print()
print("Loading scenarios for all asset classes: ")
print (line_sep)

## No senarios Loaded yet
simul_params.asset_scenarios = [None]*simul_params.num_funds

for k in range (simul_params.num_funds):
    print()
    simul_params.asset_scenarios [k] = load_asset_scenarios(
            location=SCENARIOS_FILES_LOCATION, 
            asset_class=simul_params.asset_class[k],
            n_scenarios=simul_params.num_scenarios,
            n_months=simul_params.data_size.num_columns)
    
    
print("\n>> Scenarios data ready to use...")

Asset classes:  FIXED,BALANCED,SMALL

Loading scenarios for all asset classes: 

Scenarios file :=  /Users/promise/Documents/Scenarios_File/FIXED.csv
Loading asset scenarios file from disk ...
DONE: loading asset scenarios

Scenarios file :=  /Users/promise/Documents/Scenarios_File/BALANCED.csv
Loading asset scenarios file from disk ...
DONE: loading asset scenarios

Scenarios file :=  /Users/promise/Documents/Scenarios_File/SMALL.csv
Loading asset scenarios file from disk ...
DONE: loading asset scenarios

>> Scenarios data ready to use...


In [84]:
## 17. Checking if we are loading the right data

n_months = 9 ## how many columns (months) 
n_rows  =  10 ##  ## how many rows
fund_id = 0 ## 0 means: '1rst'; 1 means: '2nd'; and 2 means: '3rd'
asset_class = simul_params.asset_class [fund_id] ## corresponding asset class 
asset_scenarios = simul_params.asset_scenarios[fund_id]
###

df= pd.DataFrame (asset_scenarios[:,: n_months+1])
df = df.applymap("{:,.5f}".format) ## show only 5 decimals
###############
print("Asset class <%s>: monthly growth factors" %(asset_class) 
      + "\n Showing first %d scenarios for the first %d month(s)"% (n_rows, n_months)) 
df.head (n_rows)


Asset class <FIXED>: monthly growth factors
 Showing first 10 scenarios for the first 9 month(s)


,0,1,2,3,4,5,6,7,8,9
0,1.00000,1.02000,1.01216,1.01470,0.98543,1.02021,1.00292,1.01789,0.97343,0.98552
1,1.00000,0.97678,0.98773,0.97936,0.98743,1.01382,1.01704,1.02590,1.01977,1.00908
2,1.00000,0.98341,0.99176,0.98127,1.00797,1.00263,1.02932,1.01575,0.99107,0.99769
3,1.00000,1.01220,1.02161,1.02563,0.99640,1.02583,1.00748,0.98956,0.99174,1.03585
4,1.00000,0.99536,1.00786,1.02879,0.99196,1.04253,1.00889,1.00168,0.99041,1.00278
5,1.00000,0.98199,1.00886,1.01330,0.98969,1.03990,1.00606,1.00759,1.00654,1.02252
6,1.00000,1.00560,1.00061,0.98621,0.98651,1.01356,1.01833,0.99060,1.00942,0.99985
7,1.00000,1.02339,0.99810,0.97294,1.01963,1.01457,1.02468,1.00647,0.99060,1.00234
8,1.00000,1.00474,0.99404,1.00239,0.98914,0.98240,0.99532,0.97371,0.99570,0.99791
9,1.00000,1.00441,1.00990,0.99732,1.00498,1.00408,1.01990,0.98994,1.00319,1.00034


In [85]:
print(asset_scenarios)

[[1.         1.01999856 1.01215969 ... 1.00013116 1.00667917 0.9906161 ]
 [1.         0.97678244 0.98772743 ... 0.99260152 0.98831659 1.0200637 ]
 [1.         0.98340643 0.99176375 ... 0.99896169 0.99585026 1.00411255]
 ...
 [1.         0.98884812 0.99488892 ... 0.99231676 1.01182299 1.03861603]
 [1.         0.99008909 1.01123712 ... 0.98003824 0.99221083 0.99094188]
 [1.         0.990227   0.98730798 ... 1.00503953 0.99789886 1.01620646]]


In [86]:
## 18. Function2 :generate  pricing parameters associated with investment fund

def investment_fund_variables(simparams):
    
    fund                =     Generic_Parameters()
    ####
    fund.cred_interest  = np.zeros(simparams.data_size.matrix_dim)
    fund.value          =np.zeros(simparams.data_size.matrix_dim)
    fund.m_exp_fee      = np.zeros(simparams.data_size.matrix_dim)
    fund.gmdb_fee       =np.zeros(simparams.data_size.matrix_dim)
    fund.gmib_fee       =np.zeros(simparams.data_size.matrix_dim)
    fund.partial_wdl    =np.zeros(simparams.data_size.matrix_dim)
    fund.total_charges  =np.zeros(simparams.data_size.matrix_dim)
    
    ###
    return fund
#...........................................................................................


## Function3:

def generate_simulation_vars_4_investment_funds (simparams):
    print(">> Generating simulation variables for all investment funds...") 
    invest_funds = [None]*simparams.num_funds ## no fund defined so far 
    tab_space = '\t'
    for k in range (simparams.num_funds):
####
       print("\tGenerating simulation variables for asset class: %s..."
             %(simul_params.asset_class[k]))
       invest_funds[k] = investment_fund_variables(simparams)
    print(">> Generating simulation variables for ...") 
    total_fund=investment_fund_variables (simparams)
    return [invest_funds, total_fund]
print("Defined function:",generate_simulation_vars_4_investment_funds)
#...........................................................................................


## Function4:

def guaranteed_fund_variables (simparams):
    fund = Generic_Parameters()
    fund.value = np.zeros(simparams.data_size.matrix_dim) 
    fund.rollup = np.zeros(simparams.data_size.matrix_dim) 
    fund.reset = np.zeros(simparams.data_size.matrix_dim)
####
    return fund



Defined function: <function generate_simulation_vars_4_investment_funds at 0x1394eca40>


In [87]:
## 19.
#For each investment fund(asset class) it will create 7(seven) 2-dimensional matrices needed 
# to do calculations of the corresponding data points
#We are using 3 asset classes + total fund ,so the code will create the total of 7x4=28 2-dimensional mattrices
# The 21 matrices will contain a total of 1,687,000x4=6,748,000 data points
#Six millions seven-hundreds and forty-eight thousands data points

## Function5:

#.......................................................................................
def generate_simulation_vars_4_guaranteed_funds(simparams):
    guar_fund = Generic_Parameters()
    print(">> Generating simulation variables for guaranteed funds...")
    ###
    print("\tGenerating simulation variables for GMDB...")
    guar_fund.gmdb= guaranteed_fund_variables(simparams)
    ###
    print("\tGenerating simulation variables for GMIB...")
    guar_fund.gmib= guaranteed_fund_variables(simparams)
    ###
    return guar_fund
    ######
#..........................................................................................      
 
# Function6:
#In oredr to have the computer generate all pricing parameters for investment funds,total funds,GMDB and GMIB

def generate_all_simulation_variables(simframework):
    line_separator = str.join('',['='*75])
    print(line_separator)
    print("Genaerating all simulation variables")
    print(line_separator)
    ####
    simparams = simframework.simul_params
    ####
    simframework.investment_funds,simframework.total_fund=\
                generate_simulation_vars_4_investment_funds(simul_params)
    ####
    simframework.guaranteed_fund = \
                    generate_simulation_vars_4_guaranteed_funds(simul_params)
    
    ####
    return True
    


    

In [88]:
## 20. Generating all 34 2-dimensional matrices

simul_framework  = Generic_Parameters()
simul_framework.simul_params = simul_params
simul_framework.policy = sample_policy

generate_all_simulation_variables(simul_framework )

print("\nDONE")

Genaerating all simulation variables
>> Generating simulation variables for all investment funds...
	Generating simulation variables for asset class: FIXED...
	Generating simulation variables for asset class: BALANCED...
	Generating simulation variables for asset class: SMALL...
>> Generating simulation variables for ...
>> Generating simulation variables for guaranteed funds...
	Generating simulation variables for GMDB...
	Generating simulation variables for GMIB...

DONE


In [89]:
## 21. Looking into the variables (matrices) and see what is inside
#The code allows to view 7 different variables(One at a time)
### Total fund value
###GMDB rollup/reset/total value
### GMIB rollup/reset/total value
#To view a specific variable uncomment the corresponding line
## For now the outputv is 0s everywhere

total_fund = simul_framework.total_fund
guaranteed_fund = simul_framework.guaranteed_fund


var_name = "TOTAL-Fund"; data = total_fund.value
#var_name = "GMDB-rollup"; data = guaranteed_fund.gmdb.rollup
#var_name = "GMDB-reset"; data = guaranteed_fund.gmdb.rollup
#var_name =  "GMDB-total" ; data = guaranteed_fund.gmdb.value


#var_name = "TOTAL-Fund"; data = total_fund.value
#var_name = "GMIB-rollup"; data = guaranteed_fund.gmib.rollup
#var_name = "GMIB-reset"; data = guaranteed_fund.gmib.rollup
#var_name =  "GMIB-total" ; data = guaranteed_fund.gmib.value

n_rows =10
n_months=20
df= pd.DataFrame(data[:,:n_months+1])

print("<%s>: value at month-end\nShowing first %d scenarios" %(var_name, n_rows) +
      " for first %d month(s)" %(n_months))
df.head (n_rows)

<TOTAL-Fund>: value at month-end
Showing first 10 scenarios for first 20 month(s)


,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [90]:
## 22. Code to view the first fund(asset class)

fund_id = 0 ## change this value in order to print a different asset class asset_class = simul_params.asset_class
asset_class= simul_params.asset_class[fund_id]
data = simul_framework.investment_funds[fund_id].value

df = pd.DataFrame(data[:, :n_months+1])
df = df.applymap("{:,.2f}".format)
print("<%s>: value at month-end; \n showing first %d scenarios "%(asset_class,n_rows) + "for first %d month(s)" % (n_months))
df.head(n_rows)
#You will notice that there is nothing inside

<FIXED>: value at month-end; 
 showing first 10 scenarios for first 20 month(s)


,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [91]:
## 23.
#Now we have the investor who come to the IC and buy a VA contract,
#and the investor gives the IC the amount of $1,000
#What the IC is going to do is:

#Put 20% in FUND-1
#Put 30 in FUND-2
#Put 50% in FUN-3
#Create a fictitious account value called: Total-FUND
#The total fund is just a "virtual account" used to keep track of aggregate values.
#........................................................
#Those transactions could be:

#Crediting interest
#Deducting M.Exp.Fee, GMDB fee, GMIB fee, partial withdrawal, deposit of additional premium (savings), etc,
#Everytime that such transactions take place, the virtual account (Total-FUND) is 
#updated so that at in point in time the IC knows what is the investor's global position.

###......................................................
##### Now processing the investor's  initial deposit(premium) of $1000 #####
#we split the 1000 into 3 funds
#ach fund received an amount prorata the corresponding fund weight.
#The fictitious fun (total fund) is also updated.
#Essentially, what the code above does is fill-in with 0s all rows of the first column of each matrix



## Function7:
def initialize_simulation_vars_4_investment_funds (simframework):
    print(">> Initializing simulation variables for all investment funds...")
####
    initial_premium = simframework.policy.initial_premium
    num_funds = simframework.simul_params.num_funds
    num_rows = simframework.simul_params.data_size.num_rows
    asset_class = simframework.simul_params.asset_class

    asset_scenarios = simframework.simul_params.asset_scenarios
####
    for k in range(num_funds):
####
        print("\tInitializing simulation variables for asset class: %s..."%(asset_class[k]))
        fund = simframework.investment_funds[k]
        fund_weight = simframework.policy.fund_weight[k] 
        fund.value[:,0] =[initial_premium*fund_weight]*num_rows

        fund.growthf = asset_scenarios [k]
        print(">>Initializing simulation variables for <total fund>:")
        simframework.total_fund.value[:,0] = [initial_premium]*num_rows   
    

In [92]:
##################################################################################
# At the end of each month,the insurance company (computer) will look at the stock prices in the market,
#calculate the growth factors with respect to the month-start and month-end stick prices,
#and once the growth factor is calculated, 
#the amount in the bank account will be automatically update to reflect the changes in the market


In [93]:
## 24. Initialization of GMDB and GMIB

##...........................................................................
##
def init_simul_vars_4_one_guaranteed_fund (gftype, simframework):
    gftype = gftype.lower()
####
    if (gftype not in ['gmdb','gmib']):
####
## TODO: print error
        return False
    print("Initializing simulation variables for:", gftype.upper())
####
    gfund = simframework.guaranteed_fund. gmdb if (gftype =='gmdb') \
                                                    else simframework.guaranteed_fund.gmib
    initial_premium = simframework.policy.initial_premium
    num_rows = simframework.simul_params.data_size.num_rows
    gfund.value [:,0]= [initial_premium]*num_rows
    gfund.rollup[:,0] = [initial_premium]*num_rows
    gfund.reset [:,0]= [initial_premium]*num_rows
####
    return True

    #return True

## 25. Initialization of guaranteed fund

#Code does initialization twice:
#1): dies it for GMDB
#2): does it for GMIB

## Function8:

def initialize_simulation_vars_4_guaranteed_funds(simframework):
    print(">> Initializing simulation variables for guaranteed funds... ") 
    init_simul_vars_4_one_guaranteed_fund("gmdb", simframework) 
    init_simul_vars_4_one_guaranteed_fund("gmib", simframework)
#................................................................................................

## Function9:

def initialize_all_simulation_variables(simframework):
    line_separator = str.join('', ['=' *75])
    print(line_separator)
    print("Initializing all simulation variables")
    print(line_separator)
    initialize_simulation_vars_4_investment_funds(simframework)
    initialize_simulation_vars_4_guaranteed_funds(simframework)
print("Defined function: ", initialize_all_simulation_variables)

Defined function:  <function initialize_all_simulation_variables at 0x12d989a80>


In [94]:
## 26.

simul_framework=Generic_Parameters()

simul_framework.simul_params = simul_params 
simul_framework.policy = sample_policy

generate_all_simulation_variables(simul_framework)
print()
initialize_all_simulation_variables(simul_framework)
print("\nDONE!")

Genaerating all simulation variables
>> Generating simulation variables for all investment funds...
	Generating simulation variables for asset class: FIXED...
	Generating simulation variables for asset class: BALANCED...
	Generating simulation variables for asset class: SMALL...
>> Generating simulation variables for ...
>> Generating simulation variables for guaranteed funds...
	Generating simulation variables for GMDB...
	Generating simulation variables for GMIB...

Initializing all simulation variables
>> Initializing simulation variables for all investment funds...
	Initializing simulation variables for asset class: FIXED...
>>Initializing simulation variables for <total fund>:
	Initializing simulation variables for asset class: BALANCED...
>>Initializing simulation variables for <total fund>:
	Initializing simulation variables for asset class: SMALL...
>>Initializing simulation variables for <total fund>:
>> Initializing simulation variables for guaranteed funds... 
Initializing s

In [95]:
####################### PROJECTIONS ########################

In [96]:
## 27.

## Function 10:

def pricing_illustration_process_partial_wdl(simfwrk,ts):
    total_fund =simfwrk.total_fund
    for k in range(simfwrk.simul_params.num_funds):
        fund = simfwrk.investment_funds[k]
        '''TO DO (LATER):Implement the partial withdrawal dynamics 
                         currently we have fund.partial_wdl :=0
        '''
    #####
     #Deduct investor's partial withdrawal from investment account
        fund.value[:,ts.ti]     -= fund.partial_wdl  [:,ts.ti]
    #### Reflect the transaction on total fund
    total_fund.value[:,ts.ti]   -= total_fund.partial_wdl[:,ts.ti]
#........................................................................................

## Function 11:

def pricing_illustration_deduct_all_charges (simfwrk,ts): 
    total_fund = simfwrk.total_fund
    for k in range(simfwrk.simul_params.num_funds):
        fund = simfwrk.investment_funds [k]
        fund.value [:,ts.ti] -= fund.total_charges[:,ts.ti]

    total_fund.value[:,ts.ti]-= total_fund.total_charges[:,ts.ti]
##.......................................................................................

## Function 12:
def pricing_illustr_update_all_fees_4_total_fund (simfwrk,ts): 
    total_fund = simfwrk.total_fund
    for k in range(simfwrk.simul_params.num_funds):
        fund = simfwrk.investment_funds [k]
#### Each opreation on any fund is reflected on the total fund total_fund.m_exp_fee [:,ts.ti] += fund.m_exp_fee


        total_fund.m_exp_fee    [:,ts.ti] += fund.gmdb_fee[:,ts.ti]
        total_fund.gmdb_fee     [:,ts.ti] += fund.gmdb_fee[:,ts.ti]
        total_fund.gmib_fee     [:,ts.ti] += fund.gmib_fee[:,ts.ti]
        total_fund.total_charges[:,ts.ti] += fund.total_charges[:,ts.ti]
#.......................................................................................

## Function 13:

def pricing_illustr__calculate_fees_4_one_rider(gftype,simfwrk,ts):
#### This function only calculates the fees, it DOES NOT deduct them!
    if (gftype == 'gmdb'):
        fee_factor = simfwrk.simul_params.gmdb.rider_fee_factor
    else:
####

       fee_factor = simfwrk.simul_params.gmib.rider_fee_factor
    for k in range (simfwrk.simul_params.num_funds):
        fund = simfwrk.investment_funds[k]
        if (gftype == 'gmdb'):
            
            fund.gmdb_fee[:,ts.ti] = fund.value[:,ts.ti_1] * fee_factor
        else:
            fund.gmib_fee[:,ts.ti] = fund.value[:,ts.ti_1] * fee_factor
#...................................................................................

### Function 14:

def pricing_illustr__calculate_rider_fees(simfwrk,ts):
    pricing_illustr__calculate_fees_4_one_rider('gmdb',simfwrk,ts)
    pricing_illustr__calculate_fees_4_one_rider('gmib',simfwrk,ts)
    
#..................................................................

### Function 15: Describing how to calculate the M.Exp.fee

def pricing_illustr__calculate_m_exp_fee(simframework,ts):
    ## fund.m_exp_fee =0 ##TO DO LATER:Do calculations
    return True

##.......................................................................................

### Function 16: Telling the computer to do all calculations

def pricing_illustration__calculate_all_fees(simfwrk,ts):
    pricing_illustr__calculate_m_exp_fee(simfwrk,ts)
    pricing_illustr__calculate_rider_fees(simfwrk,ts)
    pricing_illustr_update_all_fees_4_total_fund(simfwrk)

In [97]:
## 28.
### Function 17:

def get_guaranteed_fund_and_simul_params(gftype,simfwrk):
    if (gftype == 'gmdb'):
        gfund =simfwrk.guaranteed_fund.gmdb
        gf_params =simfwrk.simul_params.gmdb
    else:
        gfund =simfwrk.guaranteed_fund.gmib
        gf_params =simfwrk.simul_params.gmib
    #####
    return [gfund,gf_params]
    

In [98]:
## 29.

### Function 18:
def pricing_illustration___one_guaranteed_fund (gftype, simfwrk,ts):
    gfund, gf_params = get_guarenteed_fund_and_simul_params(gftype, simfwrk) ####
    total_fund = simfwrk.total_fund
##

##TODO: guaranteed fund calculated before or after the fees are charged?
##
    gfund.rollup [:,ts.ti] = gfund.rollup[:,ts.ti_1] \
        *gf_params.rollup_factor
####
    if (gf_params.is_reset_date[ts.ti]):
        gfund.reset[:,ts.ti] = \
            np.maximum(total_fund.value[:,ts.ti], gfund.reset [:,ts.ti_1])

    else:
        gfund.reset[:,ts.ti]= gfund.reset[:,ts.ti_1]
####
    gfund.value [:,ts.ti] = \
           np.maximum (gfund.rollup[:,ts.ti],  gfund.reset [:,ts.ti_1])
###..............................................................................................


### Function 19: Tells the computer to refresh the values of the GMDB and GMID guaranteed funds

def pricing_illustration_update_guaranteed_funds (simfwrk,ts): 
    pricing_illustration__one_guaranteed_fund('gmdb', simfwrk,ts) 
    pricing_illustration_one_guaranteed_fund('gmib', simfwrk,ts)
##...............................................................................................

### Function 20:Reflects the  into the total fund the interest credited to individual funds

def pricing_illustr__apply_growth_factors_2_total_fund (simfwrk,ts): 
    total_fund = simfwrk.total_fund
####
    for k in range(simfwrk.simul_params.num_funds): 
        fund = simfwrk.investment_funds [k]
####
        total_fund.itrst[:,ts.ti]+=  fund.itrst[:,ts.ti]


    total_fund.value[:,ts.ti]= total_fund.value[:,ts.ti_1] \
                                      + total_fund.itrst[:,ts.ti]
####
    total_fund.growthf[:,ts.ti] =(total_fund.value[:,ts.ti]
                                  / total_fund.value[:,ts.ti_1])
##.....................................................................................................


### Function 21: How the growth factors are applied to individual funds to determine how much interes to be credited
def pricing_illustration__apply_growth_factors (simfwrk,ts): 
    vbose_flags = simfwrk.flags.pricing_illust.timestep
####
    if (vbose_flags.print_this_iteration (ts.ti)):
####
        print("Iteration:= #%3d, "%(ts.ti)
              + "applying growth factors to investment funds...")
    for k in range(simfwrk.simul_params.num_funds):
####
        fund = simfwrk.investment_funds[k]
        fund.value [:,ts.ti] = \
                    fund.growthf[:,ts.ti] *fund.value[:,ts.ti_1]
####
        fund.itrst[:,ts.ti] = \
                   fund.value[:,ts.ti] - fund.value[:,ts.ti_1]
    pricing_illustr_apply_growth_factors_2_total_fund(simfwrk,ts)
    
#.........................................................................................................

### Function 22:Transition every month:

#1): first we use the growth factors do determine how much interest to credit to individual funds

#2) then we calculate the fees

#3): we deduct them

#4): we process the partial withdrawals

#5): and we update the values of the guaranteed funds.

def run_pricing_illustration__single_iteration(simfwrk,ts=None):
    if (ts is None):
####

#### Goto to end of month and upadte all pricing state variables 
        simfwrk.timestep.current       += 1
        simfwrk.timestep. previous     += 1
        ts                              = Generic_Parameters()
        ts.ti                           = simfwrk.timestep.current
        ts.ti_1                         = simfwrk.timestep.previous
    ####
    pricing_illustration_apply_growth_factors (simfwrk, ts)
    pricing_illustration_calculate_all_fees (simfwrk, ts)
    pricing_illustration_deduct_all_charges(simfwrk, ts)
    pricing_illustration_process_partial_wdl(simfwrk, ts)
    pricing_illustration_update_guaranteed_funds(simfwrk, ts)
##...........................................................................................


### Function 23: Tells the computer to carry out the actions mentioned above,every month
#we say that we run iterations up until the end of accumulation phase

def run_pricing_illustration (simfwrk, num_iterations=None):
    num_iter = num_iterations if not (num_iterations is None) \
                else simfwrk.simul_params.accum_phase.num_months
####
    num_iter = min(num_iter, simfwrk.simul_params.accum_phase.num_months) ####
    line_sep = str.join('',['='*65])
    print(line_sep)
    print("Pricing illustration") 
    print (line_sep)
####
    for iid in range(num_iter):
####
       run_pricing_illustration_single__iteration(simfwrk)
    return True

In [99]:
## 30.   

### Function 24: Prepares the simulation framework before we run the simulations

def init_simulation_framework_4_pricing_illustration(simfwrk):
    generate_all_simulation_variables (simfwrk)
    print()
    initialize_all_simulation_variables (simfwrk)
    print()
     ####
    simfwrk.timestep = Generic_Parameters()
    simfwrk.timestep.current =0
    simfwrk.timestep. previous=-1
####
    flags = Generic_Parameters()
    flags.pricing_illust = Generic_Parameters()

    flags.pricing_illust.timestep = Verbose_Flag_4_Timesteps() 
    flags.pricing_illust.titles = Verbose_Flag_4_Titles()
####
    simfwrk.flags=flags

In [100]:
## 31.
####### Outputs #######
total_fund = simul_framework.total_fund
guaranteed_fund = simul_framework.guaranteed_fund


#var_name = "TOTAL-Fund"; data = total_fund.value
#var_name = "GMDB-rollup"; data = guaranteed_fund.gmdb.rollup
#var_name = "GMDB-reset"; data = guaranteed_fund.gmdb.rollup
#var_name =  "GMDB-total" ; data = guaranteed_fund.gmdb.value


#var_name = "TOTAL-Fund"; data = total_fund.value
#var_name = "GMIB-rollup"; data = guaranteed_fund.gmib.rollup
#var_name = "GMIB-reset"; data = guaranteed_fund.gmib.rollup
var_name =  "GMIB-total" ; data = guaranteed_fund.gmib.value

n_rows =10
n_months=20
df= pd.DataFrame(data[:,:n_months+1])

print("<%s>: value at month-end\nShowing first %d scenarios" %(var_name, n_rows) +
      " for first %d month(s)" %(n_months))
df.head (n_rows)

<GMIB-total>: value at month-end
Showing first 10 scenarios for first 20 month(s)


,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
